In [ ]:
import numpy as np
import pandas as pd
import os
import time
import schedule
import joblib
import datetime
import functools
import multiprocessing as mp
from get_strategy_meta import get_all_strategy_meta
from query_api import get_path_dict, gen_connection, load_item_by_key, query_mysql
from get_signal_mongo import write_signal_mongo, get_signal_mongo
from mongo_functions import get_portfolio_db

# Load Strategy Metadata from Google Sheet

In [ ]:
running_on = "Windows"
output_db = "MLProduction"
output_coll = "SignalOneLive"
input_db = "Production"
input_coll = "prices"

if running_on == "Ubuntu":
    # # Running on Ubuntu Machine (OFC-THR-LWS)
    strategy_meta_df = get_all_strategy_meta(wsheet = "Accepted_Ubuntu")
else:
    # Running on Windows Machine (OFC-FOU-WS)
    # Make sure to Copy DLBatchOne_DeploymentKeyData folder to Desktop (from Z:/MLDeployment/DLBatchOne_DeploymentKeyData to C:/Users/Workstation/Desktop/DLBatchOne_DeploymentKeyData)
    # Models/scaler etc. are in C:/Users/Workstation/Desktop/DLBatchOne_DeploymentKeyData/
    strategy_meta_df = get_all_strategy_meta(wsheet = "Accepted_Windows_Local", credentials_path="C:/Users/Workstation/Desktop/DLBatchOne_DeploymentKeyData/GoogleProject/DeepLearningAlphaC.txt")
    
sdf = strategy_meta_df.loc[~strategy_meta_df["Backtest Status"].isin(["Abandon"])]
sdf["TF"] = sdf["TimeFrame"].apply(lambda x: int(x.split("_")[1]))
sdf = sdf.sort_values("TF")

# To verify if Google API (to query metadata from Google Sheet) is working
sdf.head(2)


In [ ]:
def do_mongo_1(filter_tf, output_db, output_coll, input_db, input_coll):
    # Wait 5 seconds for data to arrive.
    time.sleep(5) 
    
    # Signal inference based on latest date
    query_date = datetime.datetime.now()

    # Filter strategies
    sdf2 = sdf.loc[sdf["TF"].isin(filter_tf)]
    ordered_strat_list = sdf2["Strategy"].values
    
    for strat in ordered_strat_list:
        print("Running Strategy: ", strat)
        strategy_meta = sdf2.loc[sdf2["Strategy"] == strat].to_dict(orient = "records")[0]
        PATH_DICT = get_path_dict(strategy_meta)
        keys = []
        for k1, v1 in PATH_DICT.items():
            for k2,v2 in v1.items():
                keys.append("{}_{}".format(k1, k2))
        for key in keys:
            p = mp.Process(target=write_signal_mongo, args=(query_date, PATH_DICT, key, strategy_meta, output_db, output_coll, input_db, input_coll))
            p.start()
            p.join()
        print("Completed Strategy: ", strat)
        
def do_60_30_15_5(db_variant = "4", filter_tf = [5,15,30,60]):
    do_mongo(filter_tf)

def do_30_15_5(db_variant = "4", filter_tf = [5,15,30]):
    do_mongo(filter_tf)
        
def do_15_5(db_variant = "4", filter_tf = [5,15]):
    do_mongo(filter_tf)
        
def do_5(db_variant = "4", filter_tf = [5]):
    do_mongo(filter_tf)
    
do_mongo = functools.partial(do_mongo_1, output_db = output_db, output_coll = output_coll, input_db = input_db, input_coll = input_coll)

# Infer and Output ML Signal to MongoDB every 5 minutes

In [ ]:
schedule.every(1).hour.at(':00').do(do_60_30_15_5)
schedule.every(1).hour.at(':05').do(do_5)
schedule.every(1).hour.at(':10').do(do_5)
schedule.every(1).hour.at(':15').do(do_15_5)
schedule.every(1).hour.at(':20').do(do_5)
schedule.every(1).hour.at(':25').do(do_5)
schedule.every(1).hour.at(':30').do(do_30_15_5)
schedule.every(1).hour.at(':35').do(do_5)
schedule.every(1).hour.at(':40').do(do_5)
schedule.every(1).hour.at(':45').do(do_15_5)
schedule.every(1).hour.at(':50').do(do_5)
schedule.every(1).hour.at(':55').do(do_5)

while 1:
    schedule.run_pending()
    time.sleep(1)

# For Debugging

In [ ]:
# For Debugging
# PATH_DICT = get_path_dict(strategy_meta)
# scaler, DATA_PARAMS, MODEL_PARAMS, model = load_item_by_key(PATH_DICT, "s_VWO", exclude_model = True)
# MODEL_PARAMS

In [ ]:
%%time
# # For Debugging
# do_mongo(filter_tf = [5,15,30,60])

In [ ]:
# # For Debugging
# filter_tf = [5]
# sdf2 = sdf.loc[sdf["TF"].isin(filter_tf)]
# ordered_strat_list = sdf2["Strategy"].values
# query_date = datetime.datetime.now()
# strat = ordered_strat_list[0]
# print("Running Strategy: ", strat)
# strategy_meta = sdf2.loc[sdf2["Strategy"] == strat].to_dict(orient = "records")[0]
# PATH_DICT = get_path_dict(strategy_meta)
# key = "s_AUDUSD"
# db = get_portfolio_db("MLProduction")
# output_coll = "SignalOneBB41"
    
# input_db = "Production"
# input_coll = "prices"
# write_signal_mongo(query_date, PATH_DICT, key, strategy_meta, db, output_coll, input_db, input_coll)